<center>
    <h1>DatatableTon</h1>
    💯 datatable exercises
    <br>
    <br>
    <a href='https://github.com/vopani/datatableton/blob/master/LICENSE'>
        <img src='https://img.shields.io/badge/license-Apache%202.0-blue.svg?logo=apache'>
    </a>
    <a href='https://github.com/vopani/datatableton'>
        <img src='https://img.shields.io/github/stars/vopani/datatableton?color=yellowgreen&logo=github'>
    </a>
    <a href='https://twitter.com/vopani'>
        <img src='https://img.shields.io/twitter/follow/vopani'>
    </a>
</center>

<center>
    This is Set 10: Capstone Projects (Exercises 91-100) of <b>DatatableTon</b>: <i>💯 datatable exercises</i>
    <br>
    You can find all the exercises and solutions on <a href="https://github.com/vopani/datatableton#exercises-">GitHub</a>
</center>

**Prerequisites**
* The `datatable` package should be upgraded to the latest version (or v1.0.0+).
* The datasets from the [Avazu](https://www.kaggle.com/c/avazu-ctr-prediction/data) competition on kaggle will be used for the exercises and needs to be downloaded manually.
* It is encouraged to use your own datasets for the last exercise but you can use the datasets from the [Tradeshift](https://www.kaggle.com/c/tradeshift-text-classification) competition on kaggle for practise.

In [1]:
!python3 -m pip install -U pip
!python3 -m pip install -U datatable

     |████████████████████████████████| 1.6 MB 603 kB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.2
    Uninstalling pip-21.1.2:
      Successfully uninstalled pip-21.1.2


In [2]:
## edit the paths to the downloaded datasets of Avazu
PATH_TRAIN = '../input/avazu-ctr-prediction/train.gz'
PATH_TEST = '../input/avazu-ctr-prediction/test.gz'

import datatable as dt

**Exercise 91: Read `train.gz` and `test.gz` and assign it to `train` and `test` respectively**

In [3]:
train = dt.fread(PATH_TRAIN)
train

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,…,C17,C18,C19,C20,C21
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,1000009418151094273,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,…,1722,0,35,−1,79
1,10000169349117863715,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,…,1722,0,35,100084,79
2,10000371904215119486,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,…,1722,0,35,100084,79
3,10000640724480838376,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,…,1722,0,35,100084,79
4,10000679056417042096,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,…,2161,0,35,−1,157
5,10000720757801103869,0,14102100,1005,0,d6137915,bb1ef334,f028772b,ecad2386,7801e8d9,…,1899,0,431,100077,117
6,10000724729988544911,0,14102100,1005,0,8fda644b,25d4cfcd,f028772b,ecad2386,7801e8d9,…,2333,0,39,−1,157
7,10000918755742328737,0,14102100,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,…,2374,3,39,−1,23
8,10000949271186029916,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,…,1722,0,35,−1,79


In [4]:
test = dt.fread(PATH_TEST)
test

,id,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,…,C17,C18,C19,C20,C21
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,10000174058809263569,14103100,1005,0,235ba823,f6ebf28e,f028772b,ecad2386,7801e8d9,07d7df22,…,761,3,175,100075,23
1,10000182526920855428,14103100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,…,2616,0,35,100083,51
2,10000554139829213984,14103100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,…,2616,0,35,100083,51
3,10001094637809798845,14103100,1005,0,85f751fd,c4e18dd6,50e219e0,51cedd4e,aefc06bd,0f2161f8,…,1092,3,809,100156,61
4,10001377041558670745,14103100,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,2347f47a,f95efa07,…,2667,0,47,−1,221
5,10001521204153353724,14103100,1005,1,57fe1b20,5b626596,f028772b,ecad2386,7801e8d9,07d7df22,…,572,2,39,−1,32
6,10001911056707023378,14103100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,…,2647,2,39,100148,23
7,10001982898844213216,14103100,1005,0,85f751fd,c4e18dd6,50e219e0,388d9bfb,2347f47a,cef3e649,…,2675,3,939,100058,100
8,10002000217531288531,14103100,1005,0,543a539e,c7ca3108,3e814130,ecad2386,7801e8d9,07d7df22,…,2709,3,39,−1,23


**Exercise 92: Drop the `id` column and update the `hour` column with the hour as an integer in `train`**

In [5]:
train[:, dt.update(hour=dt.as_type(dt.str.slice(dt.as_type(dt.f.hour, str), 6, 8), int))]
del train['id']
train.head()

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,…,C17,C18,C19,C20,C21
,▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,0,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,…,1722,0,35,−1,79
1,0,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,…,1722,0,35,100084,79
2,0,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,…,1722,0,35,100084,79
3,0,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,…,1722,0,35,100084,79
4,0,0,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,07d7df22,…,2161,0,35,−1,157
5,0,0,1005,0,d6137915,bb1ef334,f028772b,ecad2386,7801e8d9,07d7df22,…,1899,0,431,100077,117
6,0,0,1005,0,8fda644b,25d4cfcd,f028772b,ecad2386,7801e8d9,07d7df22,…,2333,0,39,−1,157
7,0,0,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,…,2374,3,39,−1,23
8,1,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,…,1722,0,35,−1,79


**Exercise 93: Assign the column `id` to `test_ids`, drop it from `test` and update the `hour` column with the hour as an integer in `test`**

In [6]:
test_ids = test['id']
test_ids.head()

,id
,▪▪▪▪
0,10000174058809263569
1,10000182526920855428
2,10000554139829213984
3,10001094637809798845
4,10001377041558670745
5,10001521204153353724
6,10001911056707023378
7,10001982898844213216
8,10002000217531288531


In [7]:
test[:, dt.update(hour=dt.as_type(dt.str.slice(dt.as_type(dt.f.hour, str), 6, 8), int))]
del test['id']
test.head()

,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,…,C17,C18,C19,C20,C21
,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,0,1005,0,235ba823,f6ebf28e,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,…,761,3,175,100075,23
1,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,…,2616,0,35,100083,51
2,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,…,2616,0,35,100083,51
3,0,1005,0,85f751fd,c4e18dd6,50e219e0,51cedd4e,aefc06bd,0f2161f8,a99f214a,…,1092,3,809,100156,61
4,0,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,2347f47a,f95efa07,a99f214a,…,2667,0,47,−1,221
5,0,1005,1,57fe1b20,5b626596,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,…,572,2,39,−1,32
6,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,…,2647,2,39,100148,23
7,0,1005,0,85f751fd,c4e18dd6,50e219e0,388d9bfb,2347f47a,cef3e649,3772665a,…,2675,3,939,100058,100
8,0,1005,0,543a539e,c7ca3108,3e814130,ecad2386,7801e8d9,07d7df22,a99f214a,…,2709,3,39,−1,23


**Exercise 94: Split `train` into `valid_data`, `valid_target` for rows where `hour=23` and rest as `train_data`, `train_target`**

In [8]:
valid_data = train[dt.f.hour == 23, [col for col in train.names if col != 'click']]
valid_data.head()

,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,…,C17,C18,C19,C20,C21
,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,23,1005,0,85f751fd,c4e18dd6,50e219e0,7358e05e,b9528b13,cef3e649,c6b5fe99,…,2443,2,39,−1,23
1,23,1005,0,85f751fd,c4e18dd6,50e219e0,85b28c93,ae637522,8ded1f7a,a99f214a,…,2348,3,427,100003,61
2,23,1005,0,89a490f5,ce307e01,3e814130,ecad2386,7801e8d9,07d7df22,a99f214a,…,2227,0,679,100077,48
3,23,1005,0,d9750ee7,98572c79,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,…,1722,0,35,100084,79
4,23,1005,0,85f751fd,c4e18dd6,50e219e0,54c5d545,2347f47a,0f2161f8,9af87478,…,2480,3,297,100111,61
5,23,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,…,2502,0,35,−1,221
6,23,1005,1,5b4d2eda,16a36ef3,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,…,2227,0,679,100074,48
7,23,1005,0,85f751fd,c4e18dd6,50e219e0,45aff1a2,5c620f04,d1327cf5,a3b3a891,…,2036,3,47,−1,156
8,23,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,…,2502,0,35,100084,221


In [9]:
valid_target = train[dt.f.hour == 23, 'click']
valid_target.head()

,click
,▪
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0


In [10]:
train_data = train[dt.f.hour != 23, [col for col in train.names if col != 'click']]
train_data.head()

,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,…,C17,C18,C19,C20,C21
,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,…,1722,0,35,−1,79
1,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,…,1722,0,35,100084,79
2,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,…,1722,0,35,100084,79
3,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,…,1722,0,35,100084,79
4,0,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,07d7df22,a99f214a,…,2161,0,35,−1,157
5,0,1005,0,d6137915,bb1ef334,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,…,1899,0,431,100077,117
6,0,1005,0,8fda644b,25d4cfcd,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,…,2333,0,39,−1,157
7,0,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,…,2374,3,39,−1,23
8,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,…,1722,0,35,−1,79


In [11]:
train_target = train[dt.f.hour != 23, 'click']
train_target.head()

,click
,▪
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,1


**Exercise 95: Train a FTRL model `model_ftrl_1` using `train_data` and `train_target` and assign the predictions of `valid_data` to `preds_valid_1` and of `test` to `preds_test_1`**

In [12]:
from datatable.models import Ftrl

model_ftrl_1 = Ftrl()
model_ftrl_1.fit(train_data, train_target)
model_ftrl_1

In [13]:
preds_valid_1 = model_ftrl_1.predict(valid_data)
preds_valid_1.head()

,False,True
,▪▪▪▪,▪▪▪▪
0,0.838505,0.161495
1,0.978231,0.0217693
2,0.818862,0.181138
3,0.817515,0.182485
4,0.940199,0.0598007
5,0.808477,0.191523
6,0.871876,0.128124
7,0.973135,0.0268654
8,0.805383,0.194617


In [14]:
preds_test_1 = model_ftrl_1.predict(test)
preds_test_1.head()

,False,True
,▪▪▪▪,▪▪▪▪
0,0.865605,0.134395
1,0.828614,0.171386
2,0.798746,0.201254
3,0.950046,0.049954
4,0.694504,0.305496
5,0.709894,0.290106
6,0.786705,0.213295
7,0.88764,0.11236
8,0.936404,0.0635957


**Exercise 96: Display the feature importances of `model_ftrl_1` in descending order and calculate the logloss of `valid_target` and `preds_valid_1`**

In [15]:
model_ftrl_1.feature_importances[:, :, dt.sort(-dt.f.feature_importance)]

,feature_name,feature_importance
,▪▪▪▪,▪▪▪▪
0,C16,1
1,app_id,0.802428
2,C15,0.761605
3,site_id,0.744101
4,banner_pos,0.710922
5,C18,0.684699
6,app_domain,0.679807
7,site_category,0.671668
8,device_type,0.666143


In [16]:
preds = dt.cbind(valid_target, preds_valid_1)
preds[:, -dt.mean(dt.f.click * dt.math.log(dt.f['True']) + (1-dt.f.click) * dt.math.log(dt.f['False']))][0, 0]

0.4028581380844116

**Exercise 97: Create `submission_1` in the submission format of the competition, write it as `submission_1.csv` and submit it on Kaggle**

In [17]:
submission_1 = dt.Frame(id=test_ids, click=preds_test_1['True'])
submission_1.to_csv('submission_1.csv')
submission_1.head()

,id,click
,▪▪▪▪,▪▪▪▪
0,10000174058809263569,0.134395
1,10000182526920855428,0.171386
2,10000554139829213984,0.201254
3,10001094637809798845,0.049954
4,10001377041558670745,0.305496
5,10001521204153353724,0.290106
6,10001911056707023378,0.213295
7,10001982898844213216,0.11236
8,10002000217531288531,0.0635957


**Exercise 98: Train another FTRL model `model_ftrl_2` with `nepochs=3`, `nbins=10 ** 8`, display it's feature importances, score & evaluate it's logloss on `valid_data` and submit the predictions `preds_test_2` of `test` as `submission_2`**

In [18]:
model_ftrl_2 = Ftrl(nepochs=3, nbins=10**8)
model_ftrl_2.fit(train_data, train_target)
model_ftrl_2

In [19]:
model_ftrl_2.feature_importances[:, :, dt.sort(-dt.f.feature_importance)]

,feature_name,feature_importance
,▪▪▪▪,▪▪▪▪
0,C16,1
1,app_id,0.883275
2,C15,0.742043
3,app_domain,0.730407
4,site_id,0.727746
5,device_type,0.713567
6,site_category,0.711194
7,banner_pos,0.696404
8,C1,0.664551


In [20]:
preds_valid_2 = model_ftrl_2.predict(valid_data)
preds_valid_2.head()

,False,True
,▪▪▪▪,▪▪▪▪
0,0.823654,0.176346
1,0.98279,0.0172102
2,0.804741,0.195259
3,0.809493,0.190507
4,0.938615,0.0613849
5,0.804255,0.195745
6,0.871229,0.128771
7,0.978235,0.0217652
8,0.800372,0.199628


In [21]:
preds_test_2 = model_ftrl_2.predict(test)
preds_test_2.head()

,False,True
,▪▪▪▪,▪▪▪▪
0,0.854086,0.145914
1,0.830438,0.169562
2,0.799169,0.200831
3,0.954347,0.0456529
4,0.696285,0.303715
5,0.718953,0.281047
6,0.799779,0.200221
7,0.891158,0.108842
8,0.950826,0.0491741


In [22]:
preds = dt.cbind(valid_target, preds_valid_2)
preds[:, -dt.mean(dt.f.click * dt.math.log(dt.f['True']) + (1-dt.f.click) * dt.math.log(dt.f['False']))][0, 0]

0.39996975660324097

In [23]:
submission_2 = dt.Frame(id=test_ids, click=preds_test_2['True'])
submission_2.to_csv('submission_2.csv')
submission_2.head()

,id,click
,▪▪▪▪,▪▪▪▪
0,10000174058809263569,0.145914
1,10000182526920855428,0.169562
2,10000554139829213984,0.200831
3,10001094637809798845,0.0456529
4,10001377041558670745,0.303715
5,10001521204153353724,0.281047
6,10001911056707023378,0.200221
7,10001982898844213216,0.108842
8,10002000217531288531,0.0491741


**Exercise 99: Submit a ensemble of `model_ftrl_1` and `model_ftrl_2` by averaging the predictions as `submission_ensemble`**

In [24]:
submission_ensemble = dt.cbind(submission_1, submission_2)
submission_ensemble[:, dt.update(click = 0.5 * dt.f.click + 0.5 * dt.f['click.0'])]
del submission_ensemble[:, ['id.0', 'click.0']]
submission_ensemble.to_csv('submission_ensemble.csv')
submission_ensemble.head()

,id,click
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,10000174058809263569,0.140154
1,10000182526920855428,0.170474
2,10000554139829213984,0.201043
3,10001094637809798845,0.0478034
4,10001377041558670745,0.304605
5,10001521204153353724,0.285576
6,10001911056707023378,0.206758
7,10001982898844213216,0.110601
8,10002000217531288531,0.0563849


**Exercise 100: Use datatable on your own datasets and projects**

In [25]:
import datatable as dt
from datatable.models import Ftrl

In [26]:
train = dt.fread('../input/tradeshift-text-classification/train.csv.gz')
train.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,…,x141,x142,x143,x144,x145
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,NO,NO,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.0731394,0.481394,0.115697,0.472474,…,NO,YES,2,0.375535,0.46461
1,2,,,,,0,0,0,0,0,…,NO,YES,4,0.741682,0.59363
2,3,NO,NO,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.3418,0.0514217,0.935572,0.0414398,0.50171,…,NO,NO,1,0.776467,0.493159
3,4,YES,NO,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653912,0.0414707,0.940787,0.0908508,0.556564,…,NO,YES,3,0.168234,0.546582
4,5,NO,NO,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.41592,0,1,0,0.375297,…,NO,NO,1,0.246637,0.361045
5,6,NO,NO,cz26ErvsEb3tZqeaIlmLIwjn8D7YVXYynSLV1WltcUk=,olN1LoaeSyI8h+udI/jquozrw4R8YQ+cVwHq1dOUO5s=,1.41368,0,1,0,0.667724,…,NO,YES,2,0.263453,0.659001
6,7,NO,NO,m5L1gU5gqVRl202bsd4bnfsAkf7VyXSuHW1m02wGFBY=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,0.595554,0.0273621,0.940787,0.0908508,0.357532,…,NO,YES,2,0.200086,0.34755
7,8,NO,NO,+DmbZpPes1kiIlEhrsEbhzCI6D7BbRkQKVA93qYRvAI=,ubHy/++3FVAS97znZOt7L+cjkZFJREIiJPRZEfRIztc=,0.32578,0.0752458,0.219752,0.264429,0.409555,…,NO,NO,2,0.182557,0.397837
8,9,,,,,0,0,0,0,0,…,NO,YES,1,0.923426,0.702032


In [27]:
test = dt.fread('../input/tradeshift-text-classification/test.csv.gz')
test.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,…,x141,x142,x143,x144,x145
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1700001,YES,NO,I7K7j9mvUurktDbybGa8nYojS5TrOrQqvAandHsdjv8=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653698,0.0416845,0.940573,0.0908508,0.873563,…,NO,YES,3,0.168448,0.863581
1,1700002,NO,NO,cSuQaz0xx+UxAtskjMBN1xCeacDm/4oJYEIDFL3CMoU=,vOPAlFJxrZoxaQht2ylUA2U0jUyFGxJF7iy/SNua2+U=,1.29412,0,1,0,0.468855,…,NO,YES,1,0.694989,0.468855
2,1700003,NO,NO,VduR3ZHc2+rs/i34uA1VtOPTyOogJacJNc3mBuRRjIU=,ZSznNzP7c1xuAbA4HWA+NnJ4UXhlkZckpXtvQW/EJPw=,0.457076,0.318488,0.360094,0.444477,0.354545,…,NO,YES,0,0.326106,0.345455
3,1700004,NO,NO,S7l8SI3WbSTbhCSCcNJBWCtNjh8fSqS3ZhPZ3X+EGGU=,aZTF/lredyP4cukeN8bh6kpBjYmS1QFNpPOg2LVm3Lg=,1.08588,0.874811,0.939825,1.05473,0.767586,…,NO,YES,1,0.750832,0.760316
4,1700005,NO,NO,YKBNeCS2s8QQz1xy7Aes8Ina4f27Q022R7ggwOX58l0=,6E1RoTs+Er+giZsKw158lVBRFEjPAoSaWCcPwdXuk6k=,1.4148,0,1,0,0.348653,…,NO,NO,10,0.580717,0.337559
5,1700006,NO,NO,PvbkXzFj7ZrCmKlUrkKTSNvT9lcE2BgwQbuPWzjjr1o=,4LhhvTzxwvh2SnFtcpaRasyvph66a3YDIQCshAfyS2o=,1.29412,0,1,0,0.307239,…,NO,YES,3,0.232026,0.291246
6,1700007,,,,,0,0,0,0,0,…,NO,YES,3,0.926009,0.255943
7,1700008,NO,NO,of++PFL1bbB+kD+ut5T+Iwizb/9eFwjag8LUngoW8Gg=,EI0dNKKW6d1fYeK0fkGEp7/pY1RFj2bw/Pk47eN1Ls4=,1.4148,0,1,0,0.213946,…,NO,NO,0,0.029148,0.206022
8,1700009,NO,NO,AyUKdwJjCuwoVZ3jPrVyvnZeNS7UA/Ax6wpkgjegCGQ=,LLq6/7mBuo5pBB3xZCtooTsWqpSRfGhCln/WAHBwmNQ=,1.41368,0,1,0,0.680412,…,NO,YES,0,0.0997758,0.670103


In [28]:
train_labels = dt.fread('../input/tradeshift-text-classification/trainLabels.csv.gz')
train_labels.head()

,id,y1,y2,y3,y4,y5,y6,y7,y8,y9,…,y29,y30,y31,y32,y33
,▪▪▪▪,▪,▪,▪,▪,▪,▪,▪,▪,▪,,▪,▪,▪,▪,▪
0,1,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1
1,2,0,0,0,0,0,0,0,0,0,…,0,0,0,1,0
2,3,0,0,1,0,0,0,0,0,0,…,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1
4,5,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1
5,6,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1
6,7,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1
7,8,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1
8,9,0,0,0,0,0,1,0,0,0,…,0,0,1,0,0


In [29]:
test_ids = test['id']

del train['id']
del test['id']

submission = dt.Frame()

for target in train_labels.names[1:]:
    print(f'Model for target {target}')

    model_ftrl = Ftrl(nepochs=5, nbins=10**8, lambda1=0.1)
    model_ftrl.fit(train, train_labels[target])

    preds_test = model_ftrl.predict(test)
    submission_target = dt.Frame(id_label=test_ids[:, dt.as_type(dt.f.id, str) + f'_{target}'],
                                 pred=preds_test['True'])
    submission.rbind(submission_target)

Model for target y1
Model for target y2
Model for target y3
Model for target y4
Model for target y5
Model for target y6
Model for target y7
Model for target y8
Model for target y9
Model for target y10
Model for target y11
Model for target y12
Model for target y13
Model for target y14
Model for target y15
Model for target y16
Model for target y17
Model for target y18
Model for target y19
Model for target y20
Model for target y21
Model for target y22
Model for target y23
Model for target y24
Model for target y25
Model for target y26
Model for target y27
Model for target y28
Model for target y29
Model for target y30
Model for target y31
Model for target y32
Model for target y33


In [30]:
submission.to_csv('submission.csv')
submission.head()

,id_label,pred
,▪▪▪▪,▪▪▪▪
0,1700001_y1,1.03712e-05
1,1700002_y1,0.0001688
2,1700003_y1,7.39098e-06
3,1700004_y1,0.00121599
4,1700005_y1,0.000373244
5,1700006_y1,0.000266731
6,1700007_y1,0.222525
7,1700008_y1,1.01328e-05
8,1700009_y1,0.00172675


✅ This completes Set 10: Capstone Projects (Exercises 91-100) and concludes **DatatableTon**: *💯 datatable exercises*

Hope these exercises enabled you to learn more about the Python datatable library. If this project made you go from an amateur to a datatable ninja or helped you get a deeper understanding of datatable or was useful in teaching datatable to others, please share your thoughts and any other feedback or suggestions on our [GitHub Repository](https://github.com/vopani/datatableton) where you can also find all the exercises and solutions along with future updates.